In [23]:
import pandas as pd
import os
import json
from metrics import *

In [24]:
samples_df = pd.read_csv("/home/martin/Desktop/visdecode+/output/gemini_samples.csv")
samples_df.head()

,pred,gt
0,"{'mark_type': 'point', 'x_name': 'Year', 'y_na...","{'mark_type': 'point', 'x_name': 'Year', 'y_na..."
1,"{'mark_type': 'line', 'x_name': '', 'y_name': ...","{'mark_type': 'line', 'x_name': ' ', 'y_name':..."
2,"{'mark_type': 'pie', 'x_name': '', 'y_name': '...","{'mark_type': 'pie', 'x_name': ' ', 'y_name': ..."
3,"{'mark_type': 'pie', 'x_name': '', 'y_name': '...","{'mark_type': 'pie', 'x_name': ' ', 'y_name': ..."
4,"{'mark_type': 'hbar', 'x_name': '% of repeater...","{'mark_type': 'hbar', 'x_name': '% of repeater..."


In [25]:
mark_type_samples, x_name_samples, y_name_samples, title_samples, color_samples, title_samples = [], [], [], [], [], []
error_count = 0

for idx, row in samples_df.iterrows():

    pred = row["pred"]
    pred = pred.replace("\n","")
    pred = pred.replace("'",'"')

    gt = row["gt"]
    gt = gt.replace("\n","")
    gt = gt.replace("'",'"')

    try:

        pred_json = json.loads(pred)
        gt_json = json.loads(gt)

        mark_type_samples.append((pred_json["mark_type"], gt_json["mark_type"]))

        if gt_json["x_name"] == " ": gt_json["x_name"] = ""
        if gt_json["y_name"] == " ": gt_json["y_name"] = ""

        x_name_samples.append((pred_json["x_name"], gt_json["x_name"]))
        y_name_samples.append((pred_json["y_name"], gt_json["y_name"]))

        title_samples.append((pred_json["title"], gt_json["title"]))
        #color_samples.append((pred_json["color"], gt_json["color"]))
        title_samples.append((pred_json["title"], gt_json["title"]))

    except:
        error_count += 1
        print("JSON decode error -->", pred, gt)

error = error_count / len(samples_df)

mark_type_matrix = multiclass_confusion_matrix(mark_type_samples, MARK_TYPE_CLASSES)
mark_type_score = f1_score(mark_type_matrix, MARK_TYPE_CLASSES, average = True)

x_name_score = nld_score(x_name_samples)
y_name_score = nld_score(y_name_samples)

title_score = nld_score(title_samples)

print("\n\nERROR : \t\t", np.round(error * 100))
print("MARK-TYPE Score :\t", np.round(mark_type_score * 100))
print("X-NAME Score :\t\t", np.round(x_name_score * 100))
print("Y-NAME Score :\t\t", np.round(y_name_score * 100))
print("TITLE Score :\t\t", np.round(title_score * 100))

JSON decode error --> {"mark_type": "pie", "x_name": "", "y_name": "", "title": ""} {"mark_type": "pie", "x_name": " ", "y_name": " ", "title": nan}
JSON decode error --> {"mark_type": "pie", "x_name": "", "y_name": "", "title": ""} {"mark_type": "pie", "x_name": " ", "y_name": " ", "title": nan}
JSON decode error --> {"mark_type": "pie", "x_name": "", "y_name": "", "title": ""} {"mark_type": "pie", "x_name": " ", "y_name": " ", "title": nan}
JSON decode error --> {"mark_type": "vbar", "x_name": "Year", "y_name": "Rating (1=underdeveloped to 7=well developed)", "title": "Measure of India"s business executives perception of the quality of port infrastructure"} {"mark_type": "vbar", "x_name": "Year", "y_name": "Rating (1=underdeveloped to 7=well developed) ", "title": "Measure of India"s business executives perception of the quality of port infrastructure"}
JSON decode error --> {"mark_type": "hbar", "x_name": "Value added (constant Local Currency Unit)", "y_name": "Country", "title": "V